In [ ]:
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tiled.client import from_uri
from tqdm.notebook import tqdm

In [ ]:
c = from_uri("https://aimm.lbl.gov/api")
c

In [ ]:
list(c["NCM"])

In [ ]:
list(c["NCM"]["BM_NCMA"])

In [ ]:
node = c["NCM"]["BM_NCMA"]["Ni-L3-pristine-wanli-1"]
node

In [ ]:
pprint(dict(node.metadata))

In [ ]:
df = node.read()
df

In [ ]:
df.plot("energy", "mu_tey", label="Ni-L3")

In [ ]:
# register queries

import collections
import json
from dataclasses import dataclass

from tiled.query_registration import QueryTranslationRegistry, register


@register(name="raw_mongo", overwrite=True)
@dataclass
class RawMongo:
    """
    Run a MongoDB query against a given collection.
    """

    query: str  # We cannot put a dict in a URL, so this a JSON str.

    def __init__(self, query):
        if isinstance(query, collections.abc.Mapping):
            query = json.dumps(query)
        self.query = query

In [ ]:
fig, ax = plt.subplots()

# select all K edge measurements of pristine condition
node = c["NCM"]["BM_NCMA"].search(
    RawMongo({"metadata.charge.cycle": 0, "metadata.element.edge": "K"})
)

for k, v in node.items():
    print(k)
    df = v.read()
    if "mutrans" in df.columns:
        df.plot("energy", "mutrans", ax=ax, label=k)

In [ ]:
fig, ax = plt.subplots()

# select all Nickle L3 measurements
node = c["NCM"]["BM_NCMA"].search(
    RawMongo({"metadata.element.symbol": "Ni", "metadata.element.edge": "L3"})
)

for k, v in node.items():
    print(k)
    df = v.read()
    df.plot("energy", "mu_tey", ax=ax, label=k)

ax.set_xlim([845, 860])

In [ ]:
fig, ax = plt.subplots()

# select Manganese K edge for pristine condition for each sample
for k, s in c["NCM"].items():
    node = s.search(
        RawMongo(
            {
                "metadata.element.symbol": "Mn",
                "metadata.element.edge": "K",
                "metadata.charge.cycle": 0,
            }
        )
    )
    v = next(node.values())
    if v:
        df = v.read()
        df.plot("energy", "mutrans", ax=ax, label=k)

In [ ]:
# compare different measurements of Ni K edge of NCM materials

fig, ax = plt.subplots()

# first get a list of Ni K edges measured by Eli
eli = c["NCM"]["BM_NCMA"].search(
    RawMongo(
        {
            "metadata.element.symbol": "Ni",
            "metadata.element.edge": "K",
            "metadata.provenance.source_id": "aimm_ncm_eli",
        }
    )
)

# pick the first measurement
k,v = eli.items_indexer[0]
v.read().plot("energy", "mutrans", label=k, ax=ax)

element = v.metadata["element"]
charge = v.metadata["charge"]

# now look up corresponding measurements from chenjun
chenjun = c["NCM"]["BM_NCMA"].search(
    RawMongo(
        {
            "metadata.element.symbol": "Ni",
            "metadata.element.edge": "K",
            "metadata.provenance.source_id": "aimm_ncm_chenjun",
            "metadata.charge.cycle": charge["cycle"],
            "metadata.charge.voltage": charge["voltage"],
            "metadata.charge.state": charge["state"],
        }
    )
)

# pick the first match
k,v = chenjun.items_indexer[0]
v.read().plot("energy", "mutrans", label=k, ax=ax)

In [ ]:
# explore core dataset
list(c["core"])

In [ ]:
# search for data by provenance
node = c["core"].search(RawMongo({"metadata.provenance.source_id": "core_wanli"}))
list(node)

In [ ]:
node["MnO-1"]["O-K-1"].read().plot("energy", "mu", title="MnO (O-K)")

In [ ]:
fig, ax = plt.subplots()

# compare data from the core dataset with NCM data
df_ref = c["core"]["Ni foil  Room Temperature-1"]["Ni-K-1"].read()
df_ref["mutrans"] = np.log(df_ref["i0"] / df_ref["itrans"])
df_ref.plot("energy", "mutrans", ax=ax, label="Ni metal (newville)")

node = c["NCM"]["BM_NCMA"].search(
    RawMongo({"metadata.element.symbol": "Ni", "metadata.element.edge": "K"})
)
df = next(node.values()).read()

df.plot("energy", "mutrans", ax=ax, label="NCMA")

ax.set_title("Nickle K Edge")

In [ ]:
# export all NCM data to local hdf5 file
c["NCM"].export("NCM.h5")